# Projet 3 (LU3IN005) : Chaines de Markov et épidémiologie :
# Propagation d’une épidémie dans une population

### <u>Groupe 6 :</u>
<pr>Samy (Adem) YAZID<br>Numéro étudiant : $3811684$<br><br>Margaux MARSELOO<br>Numéro étudiant : $28630611$</pr> 

### <u>**Consignes :**</u>
<pr>

L’objectif de ce projet est de manipuler des chaînes de Markov pour étudier la propagation d’une épidémie dans une population.<br>
Notre rendu est un notebook contenant les codes commentés et les résultats interprétés.<br>
Les packages <em>numpy</em>, <em>random</em> et <em>matplotlib</em> sont utilisés.<br>
Nous allons étudier des populations constituées de $3$ types d’individus.<br>
Chaque individu est dans un des $3$ états suivant : sain $S$, infecté $I$ ou guéri $R$.
</pr>

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rd

def rec_extract_values(iterable, dico):
    """
    Paramètres:
    iterable : any
        Iterable (ou non) dont on veut extraire les valeurs dans un dictionnaire
    dico : dict
        Dictionnaire qui stockera les valeurs extraites
        
    Retourne:
    None

    Description:
    Fonction permettant d'extraire toutes les valeurs de iterable en les stockant
    dans dico, et qui compte toutes les fois où chaque valeur est apparue.
    (Dans cette fonction les strings sont considérés comme non-iterable, donc
    seront compté comme une valeur quelconque)
    
    Exemple:
    dico : {}
    rec_extract_values([1, 3, [3, 7]])
    dico : {1:1, 3:2, 7:1}
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        for l in iterable:
            rec_extract_values(l, dico)
        return dico
    else:
        if iterable not in dico:
            dico[iterable] = 1
        else:
            dico[iterable] += 1
            
def rec_sum_values(iterable):
    """
    Paramètres:
    iterable : any iterable type of (int or float)
        Iterable dont on veut sommer les valeurs
        
    Retourne:
    somme : int or float
        Somme des valeurs de iterable
    
    Description:
    Fonction permettant de retourner la somme de toutes les valeurs de iterable.
    
    Exemple:
    rec_sum_values([1, 3, [3, 7.5]]) : 14.5
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        somme = 0
        for l in iterable:
            somme += rec_sum_values(l)
        return somme
    else:
        return iterable
    
def dict_zero(dico):
    """
    Paramètres:
    dico : dict
        Dictionnaire dont on veut mettre toutes ses valeurs à 0
        
    Retourne:
    None
    
    Description:
    Fonction permettant pour un dictionnaire de conserver ses clés tout en fixant leur valeur à 0.
    
    Exemple:
    dico : {1:1, 3:2, 7:1}
    dict_zero(dico)
    dico : {1:0, 3:0, 7:0}
    """
    for key in dico:
        dico[key] = 0

## I) <u>Apprentissage des paramètres d’un modèle à partir de données</u>

<pr>
    
Dans notre modèle, nous allons considérer qu’à chaque temps :
- Chaque individu sain peut rester sain ou devenir infecté
- Chaque individu infecté peut rester infecté ou devenir guéri
- Chaque individu guéri reste guéri
et que la probabilité de passer d’un état à l’autre ne dépend que de l’état précédent.

Nous disposons d’une séquence d’observations et nous souhaitons apprendre les paramètres de la
chaîne de Markov permettant de modéliser le processus sous-jacent qui a généré la séquence.<br>
Nous avons suivi un individu pendant $10$ jours, afin de déterminer a chaque temps dans quel état se trouvait l’individu.<br>
Nous avons obtenu la séquence d’observation suivante : $S, S, S, I, I ,I ,I , I, I, R$.
</pr>


<pr><u>**Question 1:**</u> À partir de cette séquence d’observation, estimez de les probabilités de transition entre les états et dresser la matrice de probabilité de transitions.</pr>


Matrice de transition du modèle :
$
\begin{pmatrix}
  &   S &   I &   R \\
S & 2/3 & 1/3 &   0 \\
I &   0 & 5/6 & 1/6 \\
R &   0 &   0 &   1 \\
\end{pmatrix}
$

In [2]:
matrice_transition = np.array([[2/3, 1/3,   0], 
                               [  0, 5/6, 1/6], 
                               [  0,   0,   1]])

<pr>
Nous avons ensuite suivi une population de $5000$ individus, pendant $200$ jours.<br>
Pour lire les données, nous avons utiliser <code>np.loadtxt("data_exo_2022.txt")</code>.<br>
Les individus sains sont notés $0$, les infectés $1$ et les guéris $2$.

<u>**Question 2:**</u><br>
    
1) Lire des données
    
</pr>

In [3]:
data = np.loadtxt("data_exo_2022.txt")

<pr>
    
2) Estimez les probabilités de transition entre les états et dressez la matrice de probabilité de transitions
(indice pour vérifier vos résultats : la première ligne de la matrice est $[0.9308, 0.0691, 0.]$).

</pr>

In [4]:
def analyse_seq(sequence, nb_etats=-1):
    sequence = sequence.astype(int)
    
    if nb_etats == -1:
        card = {elem : 0 for elem in set(sequence)}
        nb_etats = len(card)
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
    else:
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
    
    #on détermine la fréquence
    for etat in range(1, len(sequence)):
        mat_transition_etats[sequence[etat-1]][sequence[etat]] += 1
    mat_transition_etats[sequence[etat]][sequence[etat]] += 1
        
    return mat_transition_etats  

def matrice_proba_transition_liste(liste_sequence):
    card = {}
    rec_extract_values(liste_sequence, card)
    matrice_finale = np.zeros((len(card), len(card)))
    
    for seq in liste_sequence:
        matrice_finale += analyse_seq(seq, len(card))
    print("\nMatrice des fréquences :")
    print(matrice_finale)
    print("\n")
    print("card[0] = ",card[0])
    for i in range(len(matrice_finale)):
        for j in range(len(matrice_finale[0])):
            matrice_finale[i][j] /= card[i]
            
    return matrice_finale

print("Matrice de transition de l'exemple:\n")
print(matrice_proba_transition_liste([np.array([0., 0., 0., 1., 1., 1., 1., 1., 1., 2.])]))
print("\nMatrice de transition des 5000 individus:\n")
print(matrice_proba_transition_liste(data))

Matrice de transition de l'exemple:


Matrice des fréquences :
[[2. 1. 0.]
 [0. 5. 1.]
 [0. 0. 1.]]


card[0] =  3
[[0.66666667 0.33333333 0.        ]
 [0.         0.83333333 0.16666667]
 [0.         0.         1.        ]]

Matrice de transition des 5000 individus:


Matrice des fréquences :
[[274606.  20308.      0.]
 [     0. 274803.  18953.]
 [ 16630.      0. 394700.]]


card[0] =  294914
[[0.93113925 0.06886075 0.        ]
 [0.         0.93548047 0.06451953]
 [0.04042983 0.         0.95957017]]


Matrice de fréquence finale:
$
\begin{pmatrix}
  &   S &   I &   R \\
S & 274506 & 20408 &   0 \\
I &   0 & ? & ? \\
R &   0 &   0 &  0 \\
\end{pmatrix}
$

## II) <u>Description du premier modèle</u>
<pr>La probabilité pour un individu d’être dans un de ces $3$ états au temps $t$, ne dépend que l’état dans
lequel il est au temps $t − 1$.<br>
Un individu dans l’état sain a une probabilité de $0.92$ de rester sain et une probabilité de $0.08$ de
devenir infecté. Si l’individu est infecté, il peut le rester avec une probabilité de $0.93$ et être guéri avec
une probabilité de $0.07$. S’il est dans l’état guéri, il reste dans cet état avec une probabilité de $1$.

<u>**Question 1:**</u> À partir du graphe de transition, créez la matrice de transition $A$, la matrice contenant les probabilités de transition entre les différents états. Créez une fonction permettant de vérifier qu’une matrice est stochastique.
</pr>

Matrice de transition du premier modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R &    0 &    0 &    1 \\
\end{pmatrix}
$

In [46]:
matrice_transition_modele1 = np.array([[0.92, 0.08,    0], 
                                       [   0, 0.93, 0.07], 
                                       [   0,    0,    1]])

def est_stochastique(matrice):
    epsilon = 0.00001
    for ligne in matrice:
        if abs(rec_sum_values(ligne) - 1) >= epsilon:
            return False
    return True

print("La matrice de transition de la partie I) est-elle stochastique ? :", est_stochastique(matrice_transition))

print("La matrice de transition de la partie II) est-elle stochastique ? :", est_stochastique(matrice_transition))

La matrice de transition de la partie I) est-elle stochastique ? : True
La matrice de transition de la partie II) est-elle stochastique ? : True


<pr>
Au temps $t = 0$, un individu a une probabilité de $0.9$ d’être sain et $0.1$ d’être infecté.

<u>**Question 2:**</u> Créez $π_0$ la distribution de probabilité initiale.
</pr>

$
π_0 = 
\begin{pmatrix}
   S &    I &    R \\
0.90 & 0.10 &    0 \\
\end{pmatrix}
$

In [47]:
pi_0 = np.array([0.90,
                 0.10,
                 0.00])

<pr>
Nous allons étudier l’évolution du nombre d’individu sains, infectés et guéris au cours du temps.
Dans un premier temps nous étudierons la distribution théorique, puis la distribution observée sur des simulations.
</pr>

### <u>Distribution $π_t$<u>

<pr><u>**Question 1:**</u> En utilisant $π_0$ et $A$, donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 1$ (faire d’abord le calcul à la main).
</pr>


$
π_1 = π_0 * A =
\begin{pmatrix}
   S &    I &    R \\
0.90*0.92 + 0*0 & 0.90*0.08 + 0.10*0.93 &    0.10*0.07 + 0*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.828 & 0.165 & 0.007\\
\end{pmatrix}
$

In [48]:
pi_1 = np.dot(pi_0, matrice_transition_modele1) 

print("pi_1 =", pi_1)

pi_1 = [0.828 0.165 0.007]


<pr><u>**Question 2:**</u> Donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 2$ (faire d’abord le calcul à la main).
</pr>

$
π_2 = π_1 * A =
\begin{pmatrix}
   S &    I &    R \\
0.828*0.92 + 0.007*0   &   0.828*0.08 + 0.165*0.93   &   0.165*0.07 + 0.007*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.76176 & 0.21969 & 0.01855\\
\end{pmatrix}
$

In [49]:
pi_2 = np.dot(pi_1, matrice_transition_modele1) 

print("pi_2 =", pi_2)


pi_2 = [0.76176 0.21969 0.01855]


<pr><u>**Question 3:**</u> De même pour chaque temps $t$ entre $1$ et $200$, calculez la distribution théorique des effectifs dans chaque état (<em>Rappel:</em> $π_{t+1} = π_t . A$ ).
</pr>

In [50]:
def liste_pi(pi_0, A, n):
    l_pi = [pi_0]
    for i in range(1, n+1):
        l_pi.append(np.dot(l_pi[i-1], matrice_transition_modele1))
    return l_pi

liste_pi_i = np.array(liste_pi(pi_0, matrice_transition_modele1, 200))
print("Liste des pi_i pour i entre 0 et 200 :")
print(liste_pi_i)

Liste des pi_i pour i entre 0 et 200 :
[[9.00000000e-01 1.00000000e-01 0.00000000e+00]
 [8.28000000e-01 1.65000000e-01 7.00000000e-03]
 [7.61760000e-01 2.19690000e-01 1.85500000e-02]
 [7.00819200e-01 2.65252500e-01 3.39283000e-02]
 [6.44753664e-01 3.02750361e-01 5.24959750e-02]
 [5.93173371e-01 3.33138129e-01 7.36885003e-02]
 [5.45719501e-01 3.57272330e-01 9.70081693e-02]
 [5.02061941e-01 3.75920827e-01 1.22017232e-01]
 [4.61896986e-01 3.89771324e-01 1.48331690e-01]
 [4.24945227e-01 3.99439090e-01 1.75615683e-01]
 [3.90949609e-01 4.05473972e-01 2.03576419e-01]
 [3.59673640e-01 4.08366763e-01 2.31959597e-01]
 [3.30899749e-01 4.08554980e-01 2.60545271e-01]
 [3.04427769e-01 4.06428112e-01 2.89144119e-01]
 [2.80073547e-01 4.02332365e-01 3.17594087e-01]
 [2.57667664e-01 3.96574984e-01 3.45757353e-01]
 [2.37054251e-01 3.89428148e-01 3.73517602e-01]
 [2.18089911e-01 3.81132518e-01 4.00777572e-01]
 [2.00642718e-01 3.71900434e-01 4.27456848e-01]
 [1.84591300e-01 3.61918821e-01 4.53489879e-01]
 

<pr><u>**Question 4:**</u> Représentez graphiquement la probabilité d’être dans chaque état en fonction du temps. (+ décrivez un peu ce que vous observez).
</pr>

In [80]:
def affiche_graphe_multi(liste_pi):
    #on crée les ordonnées
    liste_sains = []
    liste_infectes = []
    liste_gueris = []
    for pi in liste_pi_i:
        liste_sains.append(pi[0])
        liste_infectes.append(pi[1])
        liste_gueris.append(pi[2])
    
    #on crée les abscisses
    abscisses = [i for i in range(len(liste_pi))]
    
    #on plot le graphe comme une courbe
    plt.plot(abscisses, liste_sains)
    plt.plot(abscisses, liste_infectes)
    plt.plot(abscisses, liste_gueris)
    
    #on légende le graphe
    plt.legend(["Personnes saines", "Personnes infectées", "Personnes guéries"], loc='center right')
    
    #on nomme l'axe des abscisses
    plt.xlabel("Temps t")
    #on nomme l'axe des ordonnées
    plt.ylabel("Probabilités")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Graphique des proportions la population en fonction du temps")
       
    #on affiche le graphe
    plt.show()
    
affiche_graphe_multi(liste_pi_i)

<img src="Graph_Partie2.png"  align="bottom">

<pr><u>**Interprétation du graphique:**</u>
    Nous remarquons qu'il n'y qu'une seule vague d'infection. Les états sains et infectés sont transients tant dit que l'état guéri est récurrent. L'état guéri est en effet un état absorbant. Nous voyons bien ici que ce modèle converge rapidement vers π* = 
$ 
\begin{pmatrix}
   S &    I &     R \\
 0.0 &  0.0 & 1.0 \\
\end{pmatrix}
$
.<br>
</pr>

### <u>Tirage aléatoire des états</u>
<pr>Nous allons générer une séquence de taille $T$ en utilisant cette chaîne de Markov. Pour générer une séquence aléatoire, nous choisissons un état initial au hasard (en utilisant $π_0$) ; puis nous choisissons les états suivants en suivant les probabilités de transition <em>(= la matrice de transition $A$).</em>
Nous prennons $T=150$.
</pr>

In [83]:
def genere_sequence(pi_0, A, n):
    p = rd.random()
    sequence = []
    
    #on choisit l'état initial au hasard
    for i in range(len(pi_0)):
        if p < sum(pi_0[:i+1]):
            sequence.append(i)
            break

    #on choisit les états suivants
    for i in range(1, n):
        p = rd.random()
        for j in range(len(A)):
            if p < sum(A[sequence[-1]][:j+1]):
                sequence.append(j)
                break
    return np.array(sequence)

print("L'une des séquences aléatoires de taille 150 suivant cette chaîne de Markov est :\n")
print(genere_sequence(pi_0, matrice_transition_modele1, 150))

L'une des séquences aléatoires de taille 150 suivant cette chaîne de Markov est :

[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### <u>Modélisation d’une population</u>
<pr>Nous avons généré une séquence d’état pour un individu. Maintenant nous allons générer un ensemble de séquences pour une population de 200 individus.

<u>**Question 1:**</u> À chaque temps $t$, nous comptons le nombre d’individus sains, infectés et guéris dans la population et affichons l’évolution du nombre d’individus dans les trois états en fonction du temps.
</pr>

In [84]:
def genere_liste_sequence(pi_0, A, nb_individu, temps):
    liste_sequence = []
    for i in range(nb_individu):
        liste_sequence.append(genere_sequence(pi_0, A, temps))
    return np.array(liste_sequence)

def comptage_liste_sequence(liste_sequence, nb_etats=3):
    liste_comptage = []
    for jour in range(len(liste_sequence[0])):
        comptage = [0 for i in range(nb_etats)]
        for seq in liste_sequence:
            comptage[seq[jour]] += 1
        liste_comptage.append(comptage)
    return np.array(liste_comptage)

liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 200, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 200 séquences est :\n")
print(liste_sequence)

#EST CE QUE çA SERT D'AFFICHER çA ??
print("\n\nLa liste de 200 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 200 séquences est :

[[0 0 0 ... 2 2 2]
 [0 0 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 ...
 [0 0 0 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]]


La liste de 200 séquences compte cet effectif au cours du temps:

[[188  12   0]
 [174  22   4]
 [161  35   4]
 [153  41   6]
 [139  52   9]
 [131  55  14]
 [121  61  18]
 [115  64  21]
 [100  77  23]
 [ 89  80  31]
 [ 81  81  38]
 [ 72  84  44]
 [ 69  84  47]
 [ 65  80  55]
 [ 61  78  61]
 [ 59  73  68]
 [ 54  73  73]
 [ 47  77  76]
 [ 43  77  80]
 [ 37  79  84]
 [ 34  78  88]
 [ 32  73  95]
 [ 28  70 102]
 [ 23  74 103]
 [ 22  70 108]
 [ 21  66 113]
 [ 18  66 116]
 [ 18  59 123]
 [ 14  58 128]
 [ 14  56 130]
 [ 12  57 131]
 [ 11  54 135]
 [  9  47 144]
 [  8  47 145]
 [  8  46 146]
 [  8  43 149]
 [  8  40 152]
 [  7  36 157]
 [  6  35 159]
 [  5  35 160]
 [  5  32 163]
 [  5  31 164]
 [  4  31 165]
 [  3  30 167]
 [  2  29 169]
 [  2  27 171]
 [  2  24 174]
 [  2  23 175]
 [  2  23 175]
 [  1  22 177]
 [  1  20 179]
 [  1  19 180]

In [85]:
def affiche_graphe_nb_individu(liste_comptage):
    #on crée les ordonnées
    liste_sains = []
    liste_infectes = []
    liste_gueris = []
    
    for occurence in liste_comptage:
        liste_sains.append(occurence[0])
        liste_infectes.append(occurence[1])
        liste_gueris.append(occurence[2])
    
    #on crée les abscisses
    abscisses = [i for i in range(len(liste_comptage))]
    
    #on plot le graphe comme une courbe
    plt.plot(abscisses, liste_sains)
    plt.plot(abscisses, liste_infectes)
    plt.plot(abscisses, liste_gueris)
    
    #on légende le graphe
    plt.legend(["Personnes saines", "Personnes infectées", "Personnes guéries"], loc='center right')
    
    #on nomme l'axe des abscisses
    plt.xlabel("Temps t")
    #on nomme l'axe des ordonnées
    plt.ylabel("Nombre d'individu")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Graphique des états des individus en fonction du temps")
       
    #on affiche le graphe
    plt.show()
    
affiche_graphe_nb_individu(liste_comptage)

<img src="Graph_Partie2_2.png"  align="bottom">

<pr><u>**Question 2:**</u> Affichez le pourcentage d’individus sains infectés et guéris en fonction du temps.
</pr>

In [86]:
def pourcentage_liste_sequence(liste_sequence):
    liste_pourcentage = [[((cpt/sum(l_cpt))*100) for cpt in l_cpt] for l_cpt in comptage_liste_sequence(liste_sequence)]
    return np.array(liste_pourcentage)

liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)

##BIZARRE ! AU DEBUT ON A PAS 100% de SAINS DANS LE COMPTE ET ICI ON A ...



La liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[ 94.    6.    0. ]
 [ 87.   11.    2. ]
 [ 80.5  17.5   2. ]
 [ 76.5  20.5   3. ]
 [ 69.5  26.    4.5]
 [ 65.5  27.5   7. ]
 [ 60.5  30.5   9. ]
 [ 57.5  32.   10.5]
 [ 50.   38.5  11.5]
 [ 44.5  40.   15.5]
 [ 40.5  40.5  19. ]
 [ 36.   42.   22. ]
 [ 34.5  42.   23.5]
 [ 32.5  40.   27.5]
 [ 30.5  39.   30.5]
 [ 29.5  36.5  34. ]
 [ 27.   36.5  36.5]
 [ 23.5  38.5  38. ]
 [ 21.5  38.5  40. ]
 [ 18.5  39.5  42. ]
 [ 17.   39.   44. ]
 [ 16.   36.5  47.5]
 [ 14.   35.   51. ]
 [ 11.5  37.   51.5]
 [ 11.   35.   54. ]
 [ 10.5  33.   56.5]
 [  9.   33.   58. ]
 [  9.   29.5  61.5]
 [  7.   29.   64. ]
 [  7.   28.   65. ]
 [  6.   28.5  65.5]
 [  5.5  27.   67.5]
 [  4.5  23.5  72. ]
 [  4.   23.5  72.5]
 [  4.   23.   73. ]
 [  4.   21.5  74.5]
 [  4.   20.   76. ]
 [  3.5  18.   78.5]
 [  3.   17.5  79.5]
 [  2.5  17.5  80. ]
 [  2.5  16.   81.5]
 [  2.5  15.5  82. ]
 [  2.   15.5 

In [90]:
def pourcentage_liste_sequence(liste_comptage):
    liste_pourcentage = []
    i=0
    for sequence in liste_comptage :
        nb_individu = sum(occurence for occurence in sequence)
        liste_pourcentage.append([(occurence/nb_individu)*100 for occurence in sequence])
        
    
    return np.array(liste_pourcentage)

liste_pourcentage = pourcentage_liste_sequence(liste_comptage)
print("\n\nLa liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[100.   0.   0.]
 [100.   0.   0.]
 [100.   0.   0.]
 [100.   0.   0.]
 [100.   0.   0.]
 [100.   0.   0.]
 [ 80.  20.   0.]
 [ 40.  60.   0.]
 [ 40.  60.   0.]
 [ 40.  60.   0.]
 [ 40.  60.   0.]
 [ 20.  80.   0.]
 [ 20.  80.   0.]
 [  0.  60.  40.]
 [  0.  60.  40.]
 [  0.  40.  60.]
 [  0.  40.  60.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  80.]
 [  0.  20.  8

In [91]:
##JE NE SAIS PAS POURQUOI TA FONCTION MARCHE POUR PI ET PAS POUR CELUI LA (QUI EST PAREIL)
affiche_graphe_multi(liste_pourcentage)

ValueError: x and y must have same first dimension, but have shapes (150,) and (201,)

<pr><u>**Question 3:**</u> Quand $t$ est grand, quelle est la proportion d’individus sains, infectés et guéris ?
</pr>

<pr>
Lorsque $t$ est grand, la répartition de la population est la suivante :  π* = 
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
.<br>
</pr>

<pr><u>**Question 4:**</u> Nous avons refait les questions précédentes avec des populations de tailles différentes, $5$ individus et $5000$ individus par exemple.
</pr>

In [94]:
#Pour 5 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5 séquences est :

[[0 0 0 0 0 0 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 

In [96]:
affiche_graphe_nb_individu(liste_comptage)

/// A FAIRE AVEC L IMAGE JUSTE AU DESSUS <img src="Graph_Partie2_2.png"  align="bottom">

In [98]:
#Pour 5000 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5000, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5000 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5000 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5000 séquences est :

[[0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 ...
 [0 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]]


La liste de 5000 séquences compte cet effectif au cours du temps:

[[4475  525    0]
 [4131  834   35]
 [3770 1136   94]
 [3466 1357  177]
 [3180 1552  268]
 [2931 1691  378]
 [2709 1796  495]
 [2480 1897  623]
 [2272 1972  756]
 [2104 2007  889]
 [1943 2010 1047]
 [1765 2047 1188]
 [1632 2057 1311]
 [1489 2066 1445]
 [1378 2047 1575]
 [1273 2018 1709]
 [1164 1989 1847]
 [1066 1949 1985]
 [ 979 1895 2126]
 [ 900 1835 2265]
 [ 829 1787 2384]
 [ 771 1725 2504]
 [ 709 1668 2623]
 [ 644 1625 2731]
 [ 588 1587 2825]
 [ 547 1509 2944]
 [ 500 1456 3044]
 [ 464 1385 3151]
 [ 427 1311 3262]
 [ 385 1251 3364]
 [ 354 1192 3454]
 [ 326 1127 3547]
 [ 296 1077 3627]
 [ 276 1020 3704]
 [ 254  970 3776]
 [ 240  902 3858]
 [ 222  864 3914]
 [ 206  825 3969]
 [ 183  795 4022]
 [ 168  751 4081]
 [ 157  694 4149]
 [ 141  663 4196]
 [ 126  634 4240]
 [ 

In [99]:
liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[0.         0.         0.         ... 0.74349442 0.74349442 0.74349442]
 [0.         0.         0.         ... 0.77220077 0.77220077 0.77220077]
 [0.         0.         0.         ... 0.70671378 0.70671378 0.70671378]
 ...
 [0.         0.35971223 0.35971223 ... 0.71942446 0.71942446 0.71942446]
 [0.36630037 0.36630037 0.36630037 ... 0.73260073 0.73260073 0.73260073]
 [0.         0.         0.         ... 0.87336245 0.87336245 0.87336245]]


In [101]:
affiche_graphe_nb_individu(liste_comptage)

/// A FAIRE AVEC L IMAGE JUSTE AU DESSUS <img src="Graph_Partie2_2.png"  align="bottom">

### <u>Longueur de l’infection</u>
<pr>

<u>**Question 1:**</u> À partir des simulations, nous estimons la longueur moyenne d’une séquence de $I$.
</pr>

In [102]:
def longueur_moyenne(liste_sequence, categorie):
    moy = 0
    for seq in liste_sequence:
        for etat in seq:
            if etat == categorie:
                moy += 1
    return moy/len(liste_sequence)

longueur_moyenne_infectes = longueur_moyenne(liste_sequence, 1)

print("La longueur moyenne d'une séquence de I est de", longueur_moyenne_infectes, "jours")

La longueur moyenne d'une séquence de I est de 14.2846 jours


<u>**Question 2:**</u> Affichons la distribution observée de la longueur d’infection à l'aide d'un histogramme.
</pr>

In [103]:
def repartition_longueur_moyenne(liste_sequence, categorie):
    dico_repart = {}
    for seq in liste_sequence:
        cpt = 0
        for etat in seq:
            if etat == categorie:
                cpt += 1
        if cpt in dico_repart:
            dico_repart[cpt] += 1
        else:
            dico_repart[cpt] = 1
    for i in range(max(dico_repart)):
        if i not in dico_repart:
            dico_repart[i] = 0
    
    return dico_repart

def affiche_graphe_repartition(dico_repart):
    #on crée les abscisses
    abscisses = [key for key in dico_repart]
    abscisses.sort()
    
    #on crée les ordonnées
    ordonnees = [dico_repart[key] for key in abscisses]
    
    #on plot l'histogramme
    plt.bar(abscisses, ordonnees)
    
    #on nomme l'axe des abscisses
    plt.xlabel("Longueur de l'infection (en jours)")
    #on nomme l'axe des ordonnées
    plt.ylabel("Nombre de séquences")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Histogramme de la répartition de la longueur de l'infection")
       
    #on affiche le graphe
    plt.show()

dico_repart = repartition_longueur_moyenne(liste_sequence, 1)

#affiche_graphe_repartition(dico_repart)

<img src="Histo_Partie2.png"  align="bottom">

<u>**Question 3:**</u> Calculez la longueur théorique d’une séquence de $I$ (vous pourrez utiliser l’espérance de la loi géométrique en justifiant).
</pr>

In [104]:
p = matrice_transition_modele1[1][2]
print("L'espérance de la longueur d'une séquence de I est", (1/p))

L'espérance de la longueur d'une séquence de I est 14.285714285714285


<u>**Question 4:**</u> Comparer la longueur estimée et la longueur théorique.
</pr>

<pr>
La longueur estimée est de $14.23$ jours, tandis que la longueur théorique est de $14.29$ jours.<br>
On voit donc que les deux valeurs coïncident totalement.
</pr>

<u>**Question 5:**</u> Affichez la distribution théorique de la longueur d’infection.
</pr>

### <u>Petites modifications autour de ce premier modèle</u>
<pr>Vous pouvez maintenant modifier le modèle pour étudier différents cas de figure, en
faisant varier certains paramètres :
</pr>

<u>**Question 1:**</u> Faire varier la taille de la population.
</pr>

<u>**Question 2:**</u> Faire varier la distribution de probabilité initiale.
</pr>

<u>**Question 3:**</u> Faire varier les probabilités de transition.
</pr>

## III) <u>Description du second modèle</u>
<pr>Nous allons maintenant considérer un second modèle, les individus guéris peuvent redevenir sains avec une probabilité de $0.02$. Ils-elles peuvent perdre leur immunité face à la maladie.
</pr>

### <u>Analyse du modèle</u>

<pr>

<u>**Question 1:**</u> Ce processus peut-il être modélisé par une chaîne de Markov ?
</pr>

<pr>

<u>**Question 2:**</u>
Donnez la nouvelle matrice de transition.
</pr>

Matrice de transition du second modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R & 0.02 &    0 & 0.98 \\
\end{pmatrix}
$

In [24]:
mat_transition_modele2 = np.array([[0.92, 0.08,    0], 
                                   [   0, 0.93, 0.07], 
                                   [0.02,    0, 0.98]])

<pr>
    
<u>**Question 3:**</u> Quels est la nature des états de cette chaîne de Markov ? Est-elle périodique ? Est-elle irréductible ?
</pr>

<pr>
    
<u>**Question 4:**</u> Calculez la matrice $A × A$. À quoi correspond-elle ? Est elle stochastique ? Même question pour $A^3$ et $A^4$.
</pr>

<pr>
$
A^2 = 
\begin{pmatrix}
  &      S &      I &      R \\
S & 0.8464 &  0.148 & 0.0056 \\
I & 0.0014 & 0.8649 & 0.1337 \\
R &  0.038 & 0.0016 & 0.9604 \\
\end{pmatrix}
$
<br><br>
$
A^3 = 
\begin{pmatrix}
  &        S &        I &        R \\
S &   0.7788 & 0.205352 & 0.191569 \\
I & 0.003962 & 0.804469 &     0.07 \\
R & 0.054168 & 0.004528 & 0.941304 \\
\end{pmatrix}
$
<br><br>
$
A^4 = 
\begin{pmatrix}
  &          S &          I &          R \\
S & 0.71681296 & 0.25328136 & 0.02990568 \\
I & 0.00747642 & 0.74847313 & 0.24405045 \\
R & 0.06866064 & 0.00854448 & 0.92279488 \\
\end{pmatrix}
$
</pr>

In [25]:
A_pow_2 = np.matmul(mat_transition_modele2, mat_transition_modele2)
A_pow_3 = np.matmul(A_pow_2, mat_transition_modele2)
A_pow_4 = np.matmul(A_pow_3, mat_transition_modele2)

print("La matrice A^2 est-elle stochastique ? :", est_stochastique(A_pow_2))
print("La matrice A^3 est-elle stochastique ? :", est_stochastique(A_pow_3))
print("La matrice A^4 est-elle stochastique ? :", est_stochastique(A_pow_4))

La matrice A^2 est-elle stochastique ? : True
La matrice A^3 est-elle stochastique ? : True
La matrice A^4 est-elle stochastique ? : True


<pr>
    
<u>**Question 5:**</u> Réalisez les nouvelles simulations. Comment la population évolue-t-elle ?
</pr>

<pr>
    
<u>**Question 6:**</u> Refaites les simulations avec une autre distribution de probabilité initiale (par exemple si au temps $t = 0$, nous avons $90%$ d’infectés et $10%$ de sains). Explorez d’autres initialisations et comme pour chaque question commentez vos observations.
</pr>

<pr>
    
<u>**Question 7:**</u> Calculez la distribution de probabilité stationnaire et comparez ce résultat avec les simulations pour
$t$ assez grand.
</pr>

### <u>Longueur de l’immunité</u>

<pr>
    
On peut se demander combien de temps un individu qui a été malade, reste protégé contre la maladie.
    
<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 3:**</u> Affichez la distribution théorique et la distribution observée de la longueur de l’immunité.
</pr>

### <u>Modifier le modèle</u>

<pr>
   
<u>**Question 1:**</u> Comment l’épidémie évolue-t-elle si vous modifiez la probabilité pour un individu sain de devenir infecté ? Quelle est la nouvelle distribution a l’équilibre ?
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

## IV) <u>Confinement</u>
<pr>On peut imaginer que si des mesures de distanciation sociale sont mises en place, la probabilité de devenir infecté devient nulle.

Nous allons alterner entre les périodes de non distanciation et de distanciation :
- En période de non-confinement, nous utilisons la matrice de transition de l’exercice précédent
- En période de confinement, la probabilité de devenir infecté pour un individu sain devient nulle

<u>**Question 1:**</u> Commencez les simulations avec la matrice de transition de l’exercice précédent.
- On peut considérer qu’au temps initial tous les individus sont sains.
- Quand il y a $25%$ d’individus infectés dans la population, nous passons en période de distanciation. Le nombre d’individus infectés va décroître.
- Quand il y a moins de $10%$ d’infectés, le confinement est levé.
</pr>

<pr>
    
<u>**Question 2:**</u> Faites les simulations pour une population assez grande, représentez l’évolution du nombre d’individus à chaque temps $t$ (vous devriez voir des “vagues”), et notez les temps de confinement et de deconfinement.
</pr>

<pr>
    
<u>**Question 3:**</u> Combien de confinements / déconfinements sont nécessaires ?
</pr>

## V) <u>Discussion</u>

<pr>

<u>**Question 1:**</u> Quelles remarques critiques pouvez faire sur les modèles utilisés ?
</pr>

<pr>
    
<u>**Question 2:**</u> <em>(Optionnelle)</em> Proposez des améliorations et implémentez-les si possible.
</pr>